<a href="https://colab.research.google.com/github/IshanSFernando/Real-and-Synthetic-image-classification/blob/main/Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

train = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/randomOperated_All',
      labels = "inferred",
      label_mode = "categorical",
      color_mode = "rgb",
      batch_size = 32,
      image_size = (224,224),
      seed = 1234,
      subset = "training",
      validation_split = 0.2
)

valid = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/randomOperated_All',
      labels = "inferred",
      label_mode = "categorical",
      color_mode = "rgb",
      batch_size = 32,
      image_size = (224,224),
      seed = 1234,
      subset = "validation",
      validation_split = 0.2
)



Found 14949 files belonging to 2 classes.
Using 11960 files for training.
Found 14949 files belonging to 2 classes.
Using 2989 files for validation.


In [ ]:
classnames = train.class_names
print(classnames)
print(train)
print(valid)
num_classes = len(classnames)

['Real', 'Synthetic']
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [ ]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()


Not connected to a TPU runtime. Using CPU/GPU strategy


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

with strategy.scope():
    inputs = layers.Input(shape=(224 ,224 , 3))
    x = img_augmentation(inputs)
    outputs = EfficientNetB0(include_top=True, weights=None, classes=2)(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 224, 224, 3)      0         
 l)                                                              
                                                                 
 efficientnetb0 (Functional)  (None, 2)                4052133   
                                                                 
Total params: 4,052,133
Trainable params: 4,010,110
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
%pip install -q wandb
import wandb
from wandb.keras import WandbCallback
wandb.login()

     |████████████████████████████████| 1.8 MB 7.0 MB/s 
     |████████████████████████████████| 157 kB 62.4 MB/s 
     |████████████████████████████████| 181 kB 68.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 157 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 71.2 MB/s 
     |████████████████████████████████| 156 kB 67.2 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="efficientnet")

wandb: Currently logged in as: ishan-f. Use `wandb login --relogin` to force relogin


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

#print(tf.keras.utils.plot_model(model))
hist=model.fit(train, validation_data = valid, epochs = 100, batch_size = 32,callbacks=[WandbCallback()])


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/100
 50/374 [===>..........................] - ETA: 24:20 - loss: 3.7004 - accuracy: 0.5081

IndexError: ignored

In [ ]:
import pickle
filename="/content/drive/My Drive/images/Test.pkl"
pickle.dump(model,open(filename,'wb'))

INFO:tensorflow:Assets written to: ram://aed59e54-f692-45c0-bcb6-526db8d81a03/assets
